# Authoring Expectation Suites

## Imports

We'll start by importing everything we'll use in the notebook. A couple things to highlight:
* We still import `great_expectations` as `gx`
* We now import `great_expectations.expectations` as `gxe`. In 1.0, Expectations are top-level classes namespaced to `gxe`.
* This repo contains a `constants.py` file used across the different notebooks. We'll import the relevant files from there.

In [ ]:
import great_expectations as gx
import great_expectations.exceptions as exceptions
import great_expectations.expectations as gxe
from great_expectations.core.expectation_suite import ExpectationSuite
from great_expectations.datasource.fluent.interfaces import Datasource

from constants import (
    DB_CONNECTION_STRING,
    TABLE_NAME,
    DATASOURCE_NAME,
    ASSET_NAME,
    SUITE_NAME,
    BATCH_DEFINITION_NAME_WHOLE_TABLE,
)

### Connecting to data

Next let's connect to our data.
Datasources and DataAssets are largely unchanged in 1.0.

A new concept (that we'll touch on in greater detail later with partitioning) is the "Batch Definition", which specifies what subset of data to get from our asset.
In this example, we'll use our whole asset with the "whole table" Batch Definition.

We'll use our Batch Definition to get a concrete Batch we can validate expectations against.

In [ ]:
context = gx.get_context(mode="file")

try:
    datasource = context.sources.add_postgres(DATASOURCE_NAME, connection_string=DB_CONNECTION_STRING)
    data_asset = datasource.add_table_asset(name=ASSET_NAME, table_name=TABLE_NAME)
    batch_definition = data_asset.add_batch_definition_whole_table(BATCH_DEFINITION_NAME_WHOLE_TABLE)

    print("Created entities")

except exceptions.DataContextError:
    datasource = context.get_datasource(DATASOURCE_NAME)
    assert isinstance(datasource, Datasource)
    data_asset = datasource.get_asset(asset_name=ASSET_NAME)
    batch_definition = next(bd for bd in data_asset.batch_definitions if bd.name == BATCH_DEFINITION_NAME_WHOLE_TABLE)

    print("Untities alread exist - loaded them")

batch = data_asset.get_batch_list_from_batch_request(batch_definition.build_batch_request())[-1]  # this will get simplified

### Expectation creation

Let's create our first gx 1.0 Expectation!

Expectation classes are exposed directly in gx 1.0, and are statically typed using [Pydantic](https://docs.pydantic.dev/latest/).
Inellisense will show valid arguments and their types.

We can test the expectation against our batch without the use of a Validator, [as was needed in gx 0.18](https://docs.greatexpectations.io/docs/oss/guides/expectations/how_to_create_and_edit_expectations_with_instant_feedback_from_a_sample_batch_of_data).


In [ ]:
expectation = gxe.ExpectColumnMinToBeBetween(column="passenger_count", min_value=4, max_value=6)
batch.validate(expectation)

### Edit the expectation

The expectation is currently failing, so let's update it and verify that it succeeds.

In [ ]:
expectation.min_value = 0
batch.validate(expectation)

### Creating an Expectation Suite

Now that we have an expectation, let's create an Expectation Suite and add our expectation to it. We'll also add an expectation while we're at it.

Note that after this cell, the suite and its expectations have been persisted.

In [ ]:
try:
    suite = context.suites.add(ExpectationSuite(name=SUITE_NAME))

    suite.add_expectation(expectation)
    expectation = suite.add_expectation(gxe.ExpectColumnValuesToBeBetween(column="passenger_count", min_value=0, max_value=4))
    print("Expectation Suite created")
except exceptions.DataContextError:
    print("We've already added the suite")


### Validating an Expectation Suite

We can validate Expectation Suites against Batches the same way we validated Expectations against Batches.

In [ ]:
suite = context.suites.get(SUITE_NAME)

results = batch.validate(suite)

print(results)

### Editing an Expectation Suite

To edit an expectation suite, we find the expectation we want and edit it directly

In [ ]:
suite = context.suites.get(SUITE_NAME)
expectation = next(e for e in suite.expectations if isinstance(e, gxe.ExpectColumnValuesToBeBetween))
expectation.max_value = 10

results = batch.validate(suite)
print(results)

### Save the Expectation Suite

Now that the suite is updated, we can save it with `suite.save()`, or we can save just the individual expectation with `expectation.save()`

In [ ]:
suite.save()
expectation.save()